In [1]:
import pandas as pd
import re
from collections import namedtuple
import pdfplumber
import os 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np

### 1. Importing and  merging pop files
### 2. Adding Metropolitans and Micropolitans

In [2]:
pop_2010_2019 = pd.read_excel('../data/raw_data/TN/pop_est_2010-2019_tn.xlsx',sheet_name='Sheet1')
pop_2010_2019 = pd.DataFrame(pop_2010_2019)
pop_2010_2019 = pop_2010_2019.drop(['Census','Estimates Base',2010,2011,2012],axis=1)
pop_2010_2019.head()

,County,2013,2014,2015,2016,2017,2018,2019
0,Anderson,75299,75157,75456,75528,76056,76287,76978
1,Bedford,45565,46251,46948,47442,48211,49146,49713
2,Benton,16360,16197,16220,16082,15993,16246,16160
3,Bledsoe,13910,14501,14601,14738,14895,14883,15064
4,Blount,124874,125847,126954,128264,129999,131331,133088


In [3]:
pop_2020_2021 = pd.read_excel('../data/raw_data/TN/pop_est_2020-2021_tn.xlsx',sheet_name='Sheet1')
pop_2020_2021 = pd.DataFrame(pop_2020_2021)
pop_2020_2021 = pop_2020_2021.drop('Estimates Base',axis=1)
pop_2020_2021.head()

,County,2020,2021
0,Anderson,77209,77576
1,Bedford,50302,51119
2,Benton,15812,15872
3,Bledsoe,14930,15234
4,Blount,135566,137605


In [4]:
pop_combined = pd.merge(pop_2010_2019,pop_2020_2021,how="left",on="County")
pop_combined['County'] = pop_combined['County'].str.strip()
pop_combined['County'] = pop_combined['County'].str.upper()
pop_combined.tail(20)

,County,2013,2014,2015,2016,2017,2018,2019,2020,2021
75,SCOTT,22062,21990,21893,21890,21964,22028,22068,21842,21917
76,SEQUATCHIE,14537,14634,14641,14754,14767,14891,15026,15863,16396
77,SEVIER,93477,94660,95417,96470,97307,97895,98250,98497,99517
78,SHELBY,938363,937500,937067,936021,935251,936365,937166,928644,924454
79,SMITH,19117,19097,19308,19526,19732,19979,20157,19939,20172
80,STEWART,13277,13210,13187,13190,13422,13619,13715,13683,13855
81,SULLIVAN,156185,156404,156199,156182,156924,157599,158348,158092,159265
82,SUMNER,168754,172269,175326,179332,183756,187490,191283,197272,200557
83,TIPTON,61572,61596,61533,61223,61305,61576,61599,60961,61004
84,TROUSDALE,7810,7998,8051,9955,10831,11033,11284,11661,12035


In [5]:
metros = pd.read_excel('../data/metro_areas.xlsx',sheet_name='Filtered_States')
metros_df = pd.DataFrame(metros)
metros_tn = metros_df[metros_df['State Name']=='Tennessee']
metros_tn['County'] = metros_tn['County'].str.strip()
metros_tn['County'] = metros_tn['County'].str.upper()
metros_tn.head()

C:\Users\iulia\AppData\Local\Temp/ipykernel_32460/3293348550.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metros_tn['County'] = metros_tn['County'].str.strip()
C:\Users\iulia\AppData\Local\Temp/ipykernel_32460/3293348550.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metros_tn['County'] = metros_tn['County'].str.upper()


,CBSA Title,Metropolitan/Micropolitan Statistical Area,County,State Name
4,"Knoxville, TN",Metropolitan Statistical Area,ANDERSON,Tennessee
22,"Shelbyville, TN",Micropolitan Statistical Area,BEDFORD,Tennessee
28,"Knoxville, TN",Metropolitan Statistical Area,BLOUNT,Tennessee
30,"Cleveland, TN",Metropolitan Statistical Area,BRADLEY,Tennessee
55,"Knoxville, TN",Metropolitan Statistical Area,CAMPBELL,Tennessee


In [6]:
file ='../data/raw_data/TN/TN_2010_2022.pdf'

In [7]:
pdf = pdfplumber.open(file)
table = pdf.pages[43].extract_table()
text = pdf.pages[43].extract_text()
table = table [1:]
last_ele = table[-1]
for ele in last_ele:
     match = re.search("(Total\s\s\d{4})",ele)
     boolean = bool(match)
     year = re.findall("(?<=Total\s\s)\d{4}",ele)
     if boolean ==True:
        print(year,"Found!")

['2020'] Found!


### Extracting data for years 2013 thru 2020 and combining with pop along with Metropolitans and Micropolitans

In [8]:
pages = list(range(12,44))
tn_df = pd.DataFrame(columns=['County','fatalities','injuries','PDO','total_crashes'])
pdf = pdfplumber.open(file)
for page in pages:            
    tbl = pdf.pages[page].extract_table()
    last_ele = tbl[-1]
    df = pd.DataFrame(tbl[1::],columns=['County','fatalities','injuries','PDO','total_crashes'])
    tn_df = tn_df.append(df)    
    for ele in last_ele:
        match = re.search("(Total\s\s\d{4})",ele)
        boolean = bool(match)
        year = re.findall("(?<=Total\s\s)\d{4}",ele)
        for var in year:
             name = var         
        if boolean == True:
            name = int(name)
            tn_df = tn_df.drop(['PDO'],axis=1)
            tn_df = tn_df.iloc[:-1,:]
            tn_df['County'] = tn_df['County'].str.strip()
            tn_df['County'] = tn_df['County'].str.upper()
            df_combined_tn = pd.merge(tn_df,pop_combined[['County', name]],on="County",how='left')
            df_combined_tn_new = pd.merge(df_combined_tn,metros_tn[['CBSA Title','Metropolitan/Micropolitan Statistical Area','County']],on="County",how='left')
            df_combined_tn_new.rename(columns = {name:'population'}, inplace = True)
            df_combined_tn_new.insert(0,'Year',name)
            df_combined_tn_new.to_excel('../data/cleaned_data/%s_TN.xlsx' % name, engine='xlsxwriter',index=False)
            tn_df = pd.DataFrame(columns=['County','fatalities','injuries','PDO','total_crashes'])
        else :
            continue

### Combining all the files into a master file

In [9]:
data_file_folder ='../data/cleaned_data/'
df = []
for file in os.listdir(data_file_folder):
        if file.endswith('_TN.xlsx'):             
             df.append(pd.read_excel(os.path.join(data_file_folder,file),sheet_name='Sheet1'))
df_tn_master = pd.concat(df, axis = 0)

In [10]:
df_tn_master.isna().sum()

Year                                            0
County                                          0
fatalities                                      0
injuries                                        0
total_crashes                                   0
population                                      2
CBSA Title                                    250
Metropolitan/Micropolitan Statistical Area    250
dtype: int64

In [11]:
df_tn_master.head(10)

,Year,County,fatalities,injuries,total_crashes,population,CBSA Title,Metropolitan/Micropolitan Statistical Area
0,2013,ANDERSON,10,445,"1,769",75299.0,"Knoxville, TN",Metropolitan Statistical Area
1,2013,BEDFORD,13,287,987,45565.0,"Shelbyville, TN",Micropolitan Statistical Area
2,2013,BENTON,10,94,304,16360.0,NaN,NaN
3,2013,BLEDSOE,0,31,80,13910.0,NaN,NaN
4,2013,BLOUNT,24,734,"2,981",124874.0,"Knoxville, TN",Metropolitan Statistical Area
5,2013,BRADLEY,9,608,"2,895",101778.0,"Cleveland, TN",Metropolitan Statistical Area
6,2013,CAMPBELL,9,269,895,40247.0,"Knoxville, TN",Metropolitan Statistical Area
7,2013,CANNON,3,72,265,13700.0,"Nashville-Davidson--Murfreesboro--Franklin, TN",Metropolitan Statistical Area
8,2013,CARROLL,9,116,392,28606.0,NaN,NaN
9,2013,CARTER,6,267,"1,204",56978.0,"Johnson City, TN",Metropolitan Statistical Area


In [12]:
df_tn_master.tail(10)

,Year,County,fatalities,injuries,total_crashes,population,CBSA Title,Metropolitan/Micropolitan Statistical Area
85,2020,UNICOI,3,74,290,17867.0,"Johnson City, TN",Metropolitan Statistical Area
86,2020,UNION,9,59,187,19820.0,"Knoxville, TN",Metropolitan Statistical Area
87,2020,VANBUREN,0,20,97,6171.0,NaN,NaN
88,2020,WARREN,6,171,757,41023.0,"McMinnville, TN",Micropolitan Statistical Area
89,2020,WASHINGTON,8,580,"3,508",133080.0,"Johnson City, TN",Metropolitan Statistical Area
90,2020,WAYNE,2,54,250,16202.0,NaN,NaN
91,2020,WEAKLEY,9,110,440,32864.0,"Martin, TN",Micropolitan Statistical Area
92,2020,WHITE,4,94,427,27424.0,NaN,NaN
93,2020,WILLIAMSON,16,719,"3,853",249424.0,"Nashville-Davidson--Murfreesboro--Franklin, TN",Metropolitan Statistical Area
94,2020,WILSON,16,798,"3,250",148564.0,"Nashville-Davidson--Murfreesboro--Franklin, TN",Metropolitan Statistical Area


In [13]:
df_tn_master.to_excel('../data/TN.xlsx', engine='xlsxwriter',index=False)